# Post-hoc REPP Analysis Pipeline

Post-hoc analysis of tapping data from psynet experiments using REPP (Rhythm Extraction and Pulse Prediction) beat detection.

**Workflow**: Load participant data → Process audio (mono conversion, 44.1kHz resampling) → Extract stimulus info from CSV → Run REPP beat detection → Generate plots

**Output**: Converted audio (`.wav`), stimulus info (`.json`), and beat detection plots (`.png`) for each participant recording.


In [1]:
import os
import pandas as pd

from post_repp_pipeline import (
    setup_participant_directories,
    process_participant_audio_files,
    run_repp_analysis_for_participant,
    )

# Set Parent directory path for Assets dir
Parent directory should contain assets and data sub-dir

In [ ]:
# configure paths
base_dir = r"D:\pyspace\Djembe\psynet\data_2025\Group-1\November-2025\mali-group1-final"     # Set base directory here
output_dir = r"output"

TapTrialMusic_path = os.path.join(base_dir, "data", "TapTrialMusic.csv")
TapTrialMusic_df = pd.read_csv(TapTrialMusic_path)

print("Sub-directories of assets", os.listdir(os.path.join(base_dir, "assets")))
print("Participant Ids:", TapTrialMusic_df['participant_id'].unique())

participant_id = TapTrialMusic_df['participant_id'].unique()

## Choose sub dir and participant id

In [ ]:
choose_sub_dir = "Task 2"
choose_participant_id = 15

### convert audio and extract stimulus info per participant

In [ ]:
# Setup participant directories
participant_dir, output_participant_dir, participant_audio_fnames = setup_participant_directories(
    base_dir, choose_sub_dir, choose_participant_id, output_dir
)

os.makedirs(output_participant_dir, exist_ok=True)
# Process all audio files: convert audio and extract stimulus info
audio_stim_pairs = process_participant_audio_files(
    choose_participant_id,
    participant_dir, 
    output_participant_dir, 
    TapTrialMusic_df,
    overwrite=False  # Set to True to reprocess existing files
)

### Repp Beat Finding Analysis

In [ ]:
from custom_config import sms_tapping     # see custom_config.py
# from repp.config import sms_tapping
from repp.config import ConfigUpdater

config_params= ConfigUpdater.create_config(
    sms_tapping,    # see custom_config.py
    {
        'EXTRACT_THRESH': [0.225, 0.12],
        'EXTRACT_COMPRESS_FACTOR': 1.1,
        'EXTRACT_FIRST_WINDOW': [18, 18],           # [18, 18]
        'EXTRACT_SECOND_WINDOW': [26, 120],         # [26, 120]
        'MARKERS_MATCHING_WINDOW': 35.0,
        'TAPPING_RANGE': [200, 400], 
        'MARKERS_MAX_ERROR': 30,
        ## TODO: add a parameter that extend the MARKER ERROR THRESHOLD to 20.

    }
)

# Run REPP analysis for all recordings

results = run_repp_analysis_for_participant(
    audio_stim_pairs,
    output_participant_dir,
    config_params,
    title_plot= None,
    display_plots=False,
    figsize=(14, 12)
)


In [7]:
import os
import shutil

work_dir = r"output\Task 2"
dest_dir = r"all_pngs_Task2"

os.makedirs(dest_dir, exist_ok=True)

for root, dirs, files in os.walk(work_dir):
    for f in files:
        if f.lower().endswith(".png"):
            src = os.path.join(root, f)
            dst = os.path.join(dest_dir, f)

            # if duplicate filenames exist, rename automatically
            if os.path.exists(dst):
                base, ext = os.path.splitext(f)
                i = 1
                while os.path.exists(os.path.join(dest_dir, f"{base}_{i}{ext}")):
                    i += 1
                dst = os.path.join(dest_dir, f"{base}_{i}{ext}")

            shutil.copy2(src, dst)
